In [1]:
from datasets import load_dataset

dataset = load_dataset("spencer/dialogsum_reformat")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/diwank/.cache/huggingface/datasets/spencer___parquet/spencer--dialogsum_reformat-3db77491df58d029/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

In [15]:
import re

def to_chatml(row):
    summary = row["summary"]
    topic = row["topic"]
    sentences = row["sentences"]

    # Turn chat into chatml
    chat = [
        re.match(r"([A-Z]):\s*(.*)$", sentence.strip()).groups()
        for sentence in sentences
    ]

    chat = [
        person(utterance, name=f"Person {name}")
        for name, utterance in chat
        if name and utterance
    ]
    
    persons = list(set([g["name"] for g in chat]))

    
    chatml = [
        situation(
            f"Persons {', '.join(persons)} are talking and an AI is analyzing it."
            " At the end of the conversation, the AI will note down the topic and the summary of the conversation."
        ),
        *chat,
        thought(f"Topic: {topic}\n\nSummary:\n\n{summary}"),
    ]

    return dict(chatml=chatml)

dataset = dataset.map(to_chatml)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [16]:
dataset.push_to_hub("diwank/dialogsum-chatml", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.


...

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split dev to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]